# 8. Correct spellings in the query using edit distance

In [16]:
class PositionalIndex:
    
    def __init__(self):
        self.index = {}
        self.terms = []
        
    def buildIndex(self):
        for i in range(1, 21):
            with open(f"Data\doc{i}.txt", "r") as f:
                text = f.read()
                text = text.lower()
                text = text.split()
                self.terms.extend(text)
            for j in range(len(text)):
                if text[j] in self.index:
                    if i in self.index[text[j]]:
                        self.index[text[j]][i][0] += 1
                        self.index[text[j]][i][1].append(j)
                    else:    
                        self.index[text[j]].update({i: [1, [j]]})
                else:
                    self.index[text[j]] = {i: [1, [j]]}
            self.terms = sorted(list(set(self.terms)))

In [47]:
class Query:
    
    def __init__(self, query):
        self.query = query
        self.query = self.query.replace('(', '( ')
        self.query = self.query.replace(')', ' )')
        self.pos = PositionalIndex()
        self.pos.buildIndex()

    def spellingCorrection(self, word):
        minDist = 99999
        ans = []
        for word2 in self.pos.terms:
            mat = [[i if j==0 else j if i==0 else 0 for j in range(len(word2)+1)] for i in range(len(word)+1)]
            for i in range(1, len(word)+1):
                for j in range(1, len(word2)+1):
                    mat[i][j] = min(mat[i-1][j-1]+(0 if word[i-1]==word2[j-1] else 1), mat[i-1][j]+1, mat[i][j-1]+1)
            if mat[len(word)][len(word2)] < minDist:
                minDist = mat[len(word)][len(word2)]
                ans = [word2]
            elif mat[len(word)][len(word2)] == minDist:
                minDist = mat[len(word)][len(word2)]
                ans.append(word2)
        for i in ans:
            prompt = input(f"You searched for {word}, did you mean: {i}? (Y/N): ")
            if prompt.lower() == 'y':
                return i
        print("Cannot find term!")
        return word2

    def preProcessQuery(self):
        self.query = self.query.split()
        for i in range(len(self.query)):
            if self.query[i] not in self.pos.index:
                self.query[i] = self.spellingCorrection(self.query[i])
        self.query = ' '.join(self.query)
    
    def processPositional(self):
        if self.query == '':
            return []
        self.preProcessQuery()
        queryPos = self.query.split()
        ans = []
        if queryPos[0] in self.pos.index:
            first = self.pos.index[queryPos[0]]
        else:
            return []
                
        for docId, vals in first.items():
            count = 0
            for i in range(1, len(queryPos)):
                if queryPos[i] not in self.pos.index:
                    return []
                if docId not in self.pos.index[queryPos[i]]:
                   break
                postings1 = vals[1]
                postings2 = self.pos.index[queryPos[i]][docId][1]
                for x in postings1:
                    for y in postings2:
                        if y < x:
                            break
                        elif abs(x - y) == i:
                            count += 1
            if count == len(queryPos)-1:
                ans.append(docId)
        return ans

In [49]:
query = "artificial genaral intellegance"
q = Query(query)
ans = q.processPositional()
print(ans)

You searched for genaral, did you mean: general? (Y/N):  Y
You searched for intellegance, did you mean: intelligence? (Y/N):  Y


[11, 18]
